[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/LLaMA-VID-jupyter/blob/main/LLaMA_VID_gradio_jupyter.ipynb)

In [ ]:
%cd /content
!git clone -b dev https://github.com/camenduru/LLaMA-VID
%cd /content/LLaMA-VID

!pip install -q -e .
!pip install -q ninja
!pip install -q flash-attn --no-build-isolation

In [ ]:
import threading
import subprocess
threading.Thread(target=lambda: subprocess.run(['python', '-m', 'llamavid.serve.controller', '--host', '0.0.0.0', '--port', '10000'], check=True), daemon=True).start()
# !python -m llamavid.serve.controller --host 0.0.0.0 --port 10000

In [ ]:
import threading
import subprocess
command = [
    'python', '-m', 'llamavid.serve.model_worker',
    '--host', '0.0.0.0',
    '--controller', 'http://localhost:10000',
    '--port', '40000',
    '--worker', 'http://localhost:40000',
    '--model-path', 'YanweiLi/llama-vid-7b-full-224-long-video'
]
threading.Thread(target=lambda: subprocess.run(command, check=True, shell=False), daemon=True).start()
# !python -m llamavid.serve.model_worker --host 0.0.0.0 --controller http://localhost:10000 --port 40000 --worker http://localhost:40000 --model-path YanweiLi/llama-vid-7b-full-224-long-video

In [ ]:
!python -m llamavid.serve.gradio_web_server --controller http://localhost:10000 --model-list-mode reload --share